In [ ]:
import psycopg2
import pandas as pd

# Database connection parameters - update these!
DB_HOST = 'localhost'
DB_PORT = 5432  # default PostgreSQL port
DB_NAME = 'eduphoric'
DB_USER = 'postgres'
DB_PASS = 'password'

In [2]:
QUERY = """
SELECT
  c.table_name,
  obj_description(cls.oid) AS table_comment,
  c.column_name,
  pgd.description AS column_comment,
  c.data_type
FROM
  information_schema.columns c
JOIN
  pg_catalog.pg_class cls ON cls.relname = c.table_name
    AND cls.relkind = 'r'
LEFT JOIN
  pg_catalog.pg_description pgd ON pgd.objoid = cls.oid AND pgd.objsubid = c.ordinal_position
WHERE
  c.table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY
  c.table_schema,
  c.table_name,
  c.ordinal_position;
"""

In [3]:
conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS
        )

In [4]:
df = pd.read_sql_query(QUERY, conn)

/tmp/ipykernel_23465/2469420858.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(QUERY, conn)


In [5]:
df.head()

,table_name,table_comment,column_name,column_comment,data_type
0,absenceeventcategorydescriptor,This descriptor describes the type of absence,absenceeventcategorydescriptorid,"A unique identifier used as Primary Key, not d...",integer
1,academichonorcategorydescriptor,A designation of the type of academic distinct...,academichonorcategorydescriptorid,"A unique identifier used as Primary Key, not d...",integer
2,academicsubjectdescriptor,This descriptor holds the description of the c...,academicsubjectdescriptorid,"A unique identifier used as Primary Key, not d...",integer
3,academicweek,This entity represents the academic weeks for ...,schoolid,The identifier assigned to a school. It must b...,bigint
4,academicweek,This entity represents the academic weeks for ...,weekidentifier,The school label for the week.,character varying


In [6]:
len(df)

4637

In [7]:
input1_df = pd.read_csv('io_2/Grand Prairie- ACT.csv')
input2_df = pd.read_csv('io_2/Grand Prairie- SAT.csv')

In [8]:
from dotenv import load_dotenv
import boto3

load_dotenv()

True

In [9]:
runtime = boto3.client("bedrock-runtime")

response = runtime.converse(
    modelId="arn:aws:bedrock:us-east-1:654654390449:application-inference-profile/xjq2nc32nzby",
    messages=[{"role": "user", "content": [{"text": "when did EdFi data standard version 5.2 come out"}]}]
)
print(response["output"]["message"]["content"][0]["text"])

EdFi data standard version 5.2 was released in 2023. However, for the exact release date, I recommend checking the official EdFi Alliance website or documentation, as I want to be precise about the specific month or day of release.


In [10]:
def df_to_markdown_table(df: pd.DataFrame) -> str:
    if df.empty:
        return "No results found."

    headers = df.columns.tolist()
    md_table = "| " + " | ".join(headers) + " |\n"
    md_table += "|" + "|".join(["---"] * len(headers)) + "|\n"

    for row in df.itertuples(index=False, name=None):
        md_table += "| " + " | ".join(map(str, row)) + " |\n"

    return md_table

In [11]:
print(df_to_markdown_table(input1_df[['Field', 'Description', 'Sample']]))


| Field | Description | Sample |
|---|---|---|
| ACT ID | Unique ACT student identifier | 999999999 |
| Last Name | Student’s last name | LAST |
| First Name | Student’s first name | FIRST |
| Middle Initial | Student’s middle name initial | M |
| Last Updated (CT) | Date the student’s test record was last updated (Central Time) | 3/24/2025 |
| Test Date | Date the ACT test was taken | 25-Mar |
| DOB | Student’s date of birth | 2/18/2008 |
| ACT composite score | Overall ACT composite score (average of all sections) | 15 |
| ACT math score | Score on the Math section of the ACT | 16 |
| ACT science score | Score on the Science section of the ACT | 17 |
| ACT STEM score | Combined Math and Science score representing STEM readiness | 17 |
| ACT English score | Score on the English section of the ACT | 15 |
| ACT reading score | Score on the Reading section of the ACT | 12 |
| State Org Number | State-level organization code | 4925506 |
| District Org Number | District-level organization 

In [24]:
print(df_to_markdown_table(input2_df[['Field', 'Description', 'Sample']]))

| Field | Description | Sample |
|---|---|---|
| SID | Student internal system ID | 1000000000 |
| TX Unique Student ID | State-assigned Texas unique student ID | 9999999999 |
| Local ID | Local district-assigned student ID | 1 |
| Last Name | Student’s last name | Last |
| First Name | Student’s first name | First |
| Grade Level | Student’s current grade | 12 |
| Campus | Student’s campus name and code | (057910010) - John A Dubiski Career H S |
| SAT Test Date | Date the SAT was taken | 10/5/2024 |
| Total Score | Total SAT composite score | 1050 |
| Evidence-Based Reading and Writing Section Score | Combined score for Reading and Writing & Language sections | 530 |
| Math Test Score | Overall Math section score | 520 |
| Reading Test Score | Score for the Reading subsection | nan |
| Writing and Language Test Score | Score for the Writing and Language subsection | nan |
| Math Test Score (Repeated) | Duplicate of Math Test Score field (possibly a formatting error) | nan |
| Command

In [12]:
import re
import ast

def parse_llm_response_method(response_text):
    list_match = re.search(r'\[.*\]', response_text, re.DOTALL)
    if list_match:
        list_str = list_match.group(0)
        try:
            return ast.literal_eval(list_str)
        except (ValueError, SyntaxError) as e:
            print(f"AST parsing failed: {e}")
            return None
    return None

In [ ]:
def call_llm(input_df, df_chunk):
    
    md_input = df_to_markdown_table(input_df[['Field', 'Description', 'Sample']])
    md_reference = df_to_markdown_table(df_chunk[['table_name', 'table_comment', 'column_name', 'column_comment', 'data_type']])
    
    prompt = f"""
    For each field, description and sample below, find a matching table_name and column_name from the provided chunk, use the description, data type for this.
    If you are not confident leave it blank.
    
    Table is:
    {md_input}

    Chunk:
    {md_reference}
    Return as a list of dicts: field, mapped_table, mapped_column.

    If you cannot find a match, return an empty string for mapped_table and mapped_column.

    Example output:
    [
        {{'field': 'Field1', 'mapped_table': 'table1', 'mapped_column': 'column1'}},
        {{'field': 'Field2', 'mapped_table': '', 'mapped_column': ''}}
    ]
    """

    # response = runtime.converse(
    #     modelId="arn:aws:bedrock:us-east-1:654654390449:application-inference-profile/s0naya29upmr",
    #     messages=[{"role": "user", "content": [{"text": prompt}]}]
    # )
    response = runtime.converse(
        modelId="arn:aws:bedrock:us-east-1:654654390449:application-inference-profile/xjq2nc32nzby",
        messages=[{"role": "user", "content": [{"text": prompt}]}]
    )
    parsed_reponse = parse_llm_response_method(response["output"]["message"]["content"][0]["text"])
    return parsed_reponse if parsed_reponse else []

In [14]:
print(input1_df.columns)

Index(['Field', 'Description', 'Identity', 'Required', 'Sample', 'Format'], dtype='object')


In [15]:
print(df.iloc[0:100].columns)

Index(['table_name', 'table_comment', 'column_name', 'column_comment',
       'data_type'],
      dtype='object')


In [16]:
call_llm(input1_df, df.iloc[0:100])

[{'field': 'ACT ID', 'mapped_table': '', 'mapped_column': ''},
 {'field': 'Last Name', 'mapped_table': '', 'mapped_column': ''},
 {'field': 'First Name', 'mapped_table': '', 'mapped_column': ''},
 {'field': 'Middle Initial', 'mapped_table': '', 'mapped_column': ''},
 {'field': 'Last Updated (CT)',
  'mapped_table': 'assessment',
  'mapped_column': 'lastmodifieddate'},
 {'field': 'Test Date',
  'mapped_table': 'assessmentadministration',
  'mapped_column': 'administrationidentifier'},
 {'field': 'DOB', 'mapped_table': '', 'mapped_column': ''},
 {'field': 'ACT composite score',
  'mapped_table': 'assessment',
  'mapped_column': 'maxrawscore'},
 {'field': 'ACT math score',
  'mapped_table': 'assessmentacademicsubject',
  'mapped_column': 'academicsubjectdescriptorid'},
 {'field': 'ACT science score',
  'mapped_table': 'assessmentacademicsubject',
  'mapped_column': 'academicsubjectdescriptorid'},
 {'field': 'ACT STEM score', 'mapped_table': '', 'mapped_column': ''},
 {'field': 'ACT Englis

In [17]:
def process_chunk_results(input_df, df, chunk_size=150):
    """
    Process chunks and handle duplicate field mappings by creating 
    'possible other options' for fields that have multiple mappings.
    """
    results = []
    field_mappings = {}  # Track existing mappings for each field
    
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i + chunk_size]
        dictionary = call_llm(input_df, chunk)
        
        # Process each mapping from this chunk
        for mapping in dictionary:
            field = mapping['field']
            mapped_table = mapping['mapped_table']
            mapped_column = mapping['mapped_column']
            
            # Skip if mapped_column is empty
            if not mapped_column or mapped_column.strip() == '':
                continue
            
            # Check if this field already exists in results
            if field in field_mappings:
                # Field already exists, add to possible other options
                existing_idx = field_mappings[field]['index']
                
                # Initialize 'possible_other_options' if it doesn't exist
                if 'possible_other_options' not in results[existing_idx]:
                    results[existing_idx]['possible_other_options'] = []
                
                # Add the new mapping as a tuple (table_name, column_name)
                new_option = (mapped_table, mapped_column)
                if new_option not in results[existing_idx]['possible_other_options']:
                    results[existing_idx]['possible_other_options'].append(new_option)
            else:
                # First time seeing this field, add it to results
                new_entry = {
                    'field': field,
                    'mapped_table': mapped_table,
                    'mapped_column': mapped_column
                }
                results.append(new_entry)
                
                # Track this field and its index in results
                field_mappings[field] = {
                    'index': len(results) - 1,
                    'table': mapped_table,
                    'column': mapped_column
                }
    
    return results

In [18]:
result = process_chunk_results(input2_df, df, chunk_size=200)

In [19]:
result

[{'field': 'Grade Level',
  'mapped_table': 'assessment',
  'mapped_column': 'assessedgradelevel',
  'possible_other_options': [('bellschedulegradelevel',
    'gradeleveldescriptorid'),
   ('courseofferedgradelevel', 'gradeleveldescriptorid'),
   ('credentialgradelevel', 'gradeleveldescriptorid'),
   ('educationcontentappropriategradelevel', 'gradeleveldescriptorid'),
   ('gradeleveldescriptor', 'gradeleveldescriptorid'),
   ('interventionappropriategradelevel', 'gradeleveldescriptorid'),
   ('learningstandardgradelevel', 'gradeleveldescriptorid'),
   ('openstaffpositioninstructionalgradelevel', 'gradeleveldescriptorid'),
   ('sectionofferedgradelevel', 'gradeleveldescriptorid'),
   ('studentassessment', 'whenassessedgradeleveldescriptorid'),
   ('studentassessmentregistration', 'assessmentgradeleveldescriptorid'),
   ('studentschoolassociation', 'gradelevel')]},
 {'field': 'SAT Test Date',
  'mapped_table': 'assessmentadministrationperiod',
  'mapped_column': 'begindate',
  'possible_

In [20]:
def results_to_dataframe(results):
    """Convert results to a pandas DataFrame"""
    df_data = []
    for result in results:
        row = {
            'field': result['field'],
            'mapped_table': result['mapped_table'],
            'mapped_column': result['mapped_column'],
            'possible_other_options': result.get('possible_other_options', [])
        }
        df_data.append(row)
    
    return pd.DataFrame(df_data)

In [21]:
output_df = results_to_dataframe(result)

In [22]:
len(output_df)

23

In [23]:
output_df.to_csv('io_2/output - Grand SAT.csv', index=False)